Загрузим данные (к сожалению, я взяла только 5000 отзывов, потому что на обработку большего количества не было времени). Категория Grocery and Gourme Food, данные можно скачать по ссылке https://jmcauley.ucsd.edu/data/amazon_v2/categoryFilesSmall/Grocery_and_Gourmet_Food_5.json.gz

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [42]:
data_path = "drive/MyDrive/avtobr/food_data.json.gz"
my_folder = "drive/MyDrive/avtobr/"

In [4]:
import gzip
import json

def data_parse(path):
    js_data = []
    g = gzip.open(path, 'rb')
    for l in g:
        js_data.append(json.loads(l))
    return js_data

data = data_parse(data_path)

In [17]:
data_big = data

In [18]:
data = data[:5000]

In [ ]:
for i in data[150:155]:
    print(i)

Методы выделения упоминаний:

1) через wordnet: берем гипонимы слов "еда", "напиток" и др., потом можно взять гипонимы гипонимов (или даже сделать несколько таких итераций) и получить список разной возможной еды. Это быстро, но скорее всего мы так потеряем большую часть вариантов, потому что еду обычно называют по типу продукта (чай, паста, салат и т.д.), и наименований слишком много, чтобы этот метод достал все

2) похожий способ, но через word2vec: обучить модель на корпусе отзывов и найти самые похожие слова через близость к слову "еда" и похожим.

3) Можно сделать шаблоны вроде "this NOUN", так как кажется в отзывах люди часто упоминают название объекта именно в контекстах вроде "этот чай очень вкусный". Также это можно комбинировать с методом w2v, чтобы отфильтровать слова, попавшие туда по ошибке


Реализовывать буду вариант 3

In [ ]:
!pip install stanza

import re
import stanza
from tqdm import tqdm
stanza.download('en')
nlp = stanza.Pipeline('en', processors='tokenize,lemma,pos', tokenize_no_ssplit=True)

 Для начала найдём названия: всё, что стоит после "this is|this|these", при этом дальше идёт проверка части речи, и перед существительным может идти сколько угодно артиклей и прилагательных, это поможет найт больше названий.

In [13]:
entities = []

for review in tqdm(data):
    if 'reviewText' in review:
        rev_text = review['reviewText']
        reg_res = [res[1] for res in re.findall(r'(this is|this|these) ([\w\s]+)', rev_text)]
        for res in reg_res:
            group = [(word.text, word.upos) for sent in nlp(res).sentences for word in sent.words]
            for word in group:
                noun_phrase = []
                if word[1] == 'ADJ' or word[1] == 'DET':
                    continue
                elif word[1] == 'NOUN':
                    entities.append(word[0])
                else:
                    break

100%|██████████| 5000/5000 [06:36<00:00, 12.61it/s]


In [14]:
ent_list = list(set(entities))

In [79]:
ent_list = [el.lower() for el in ent_list]

теперь подключим w2v, попробуем удалить несовпадающие

In [5]:
import gensim
import logging
import urllib.request

import warnings
warnings.filterwarnings('ignore')

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [20]:
reviews = [review['reviewText'] for review in data if 'reviewText' in review]

In [78]:
with open(my_folder + 'ent_list.txt', 'r', encoding='utf-8') as f:
    ent_list = list(f.read()[2:-2].split('\', \''))

In [43]:
with open(my_folder + 'reviews.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(reviews))

In [48]:
with open(my_folder + 'ent_list.txt', 'w', encoding='utf-8') as f:
    f.write(str(ent_list))

In [49]:
w2v_data = gensim.models.word2vec.LineSentence(my_folder + 'reviews.txt')

In [52]:
model_food = gensim.models.Word2Vec(w2v_data, size=300, window=5, min_count=1)

In [62]:
#ent_list.remove(antimatch)
antimatch = model_food.wv.doesnt_match(ent_list)
print(antimatch)

quinoa


Получилось довольно плохо, в лишних довольно быстро начали появляться названия продуктов, поэтому в результате отфильтровать получилось плохо.

Теперь найдём все биграммы с этими словами

In [152]:
from string import punctuation
left_context = []
right_context = []

for review in reviews:
    review = review.translate(str.maketrans('', '', punctuation)).lower()
    review = review.split()
    for i, word in enumerate(review):
        if word in ent_list:
            if i > 0:
                left_context.extend([review[i - 1], review[i], '|'])
            if i + 1 < len(review):
                right_context.extend([review[i], review[i + 1], '|'])

In [ ]:
import nltk
nltk.download('stopwords')
ignored_words = nltk.corpus.stopwords.words('english') + ['|']
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()

С помощью nltk посмотрим, как выделяют коллокации три различные метрики:

(стоп-слова, артикли и т.д. удаляются на этом этапе)

In [154]:
l_finder = BigramCollocationFinder.from_words(left_context)
l_finder.apply_word_filter(lambda w: w.lower() in ignored_words)

print('pmi:', *l_finder.nbest(bigram_measures.pmi, 20))
print('student_t:', *l_finder.nbest(bigram_measures.student_t, 20))
print('raw_freq:', *l_finder.nbest(bigram_measures.raw_freq, 20))

print('\n')

r_finder = BigramCollocationFinder.from_words(right_context)
r_finder.apply_word_filter(lambda w: w.lower() in ignored_words)
print('pmi:', *r_finder.nbest(bigram_measures.pmi, 20))
print('student_t:', *r_finder.nbest(bigram_measures.student_t, 20))
print('raw_freq:', *r_finder.nbest(bigram_measures.raw_freq, 20))

pmi: [('caviar', 'sourkrout'), ('mortal', 'coil'), ('splendid', 'potion'), ('choking', 'stage'), ('discovered', 'aidells'), ('dingy', 'colours'), ('index', 'rates'), ('teasthis', 'wording'), ('pay', '6box'), ('british', 'confections'), ('flagitious', 'concoction'), ('trying', 'quinoa'), ('chronic', 'allergies'), ('gumball', 'category'), ('reviewer', 'ointment'), ('rockthese', 'meltaways'), ('web', 'page'), ('british', 'delights'), ('childhood', 'beverage'), ('evening', 'beverage')]
student_t: [('hot', 'sauce'), ('great', 'product'), ('hot', 'sauces'), ('black', 'tea'), ('vanilla', 'extract'), ('iced', 'tea'), ('ghost', 'pepper'), ('great', 'price'), ('good', 'price'), ('good', 'product'), ('great', 'flavor'), ('good', 'flavor'), ('celestial', 'seasonings'), ('good', 'quality'), ('great', 'taste'), ('excellent', 'product'), ('green', 'curry'), ('corn', 'syrup'), ('green', 'tea'), ('davids', 'cookies')]
raw_freq: [('hot', 'sauce'), ('great', 'product'), ('hot', 'sauces'), ('black', 'tea'

Мне показалось, что правый контекст как правило не очень информативен, а pmi выделяет много редких (зачастую случайных) сочетаний, поэтому возьмём t-score:

In [156]:
for entity in ['tea', 'sauce', 'product', 'drink', 'stuff']:
    ent_left_context = []
    for review in reviews:
        review = review.translate(str.maketrans('', '', punctuation)).lower()
        review = review.replace('the ', '').replace(' a ', ' ')
        review = review.split()
        for i, word in enumerate(review):
            if word == entity:
                if i > 0:
                    ent_left_context.extend([review[i - 1], review[i], '|'])
    finder = BigramCollocationFinder.from_words(ent_left_context)
    finder.apply_word_filter(lambda w: w.lower() in ignored_words)
    print(entity, 'student_t:', *finder.nbest(bigram_measures.student_t, 10))

tea student_t: ('black', 'tea') ('iced', 'tea') ('green', 'tea') ('chai', 'tea') ('good', 'tea') ('hot', 'tea') ('ice', 'tea') ('lipton', 'tea') ('peach', 'tea') ('favorite', 'tea')
sauce student_t: ('hot', 'sauce') ('soy', 'sauce') ('insanity', 'sauce') ('pepper', 'sauce') ('seasoning', 'sauce') ('bbq', 'sauce') ('hottest', 'sauce') ('chili', 'sauce') ('death', 'sauce') ('wing', 'sauce')
product student_t: ('great', 'product') ('good', 'product') ('excellent', 'product') ('quality', 'product') ('awesome', 'product') ('nice', 'product') ('ok', 'product') ('organic', 'product') ('vanilla', 'product') ('wonderful', 'product')
drink student_t: ('dont', 'drink') ('cold', 'drink') ('cant', 'drink') ('morning', 'drink') ('rather', 'drink') ('delicious', 'drink') ('favorite', 'drink') ('never', 'drink') ('rarely', 'drink') ('bedtime', 'drink')
stuff student_t: ('good', 'stuff') ('great', 'stuff') ('hot', 'stuff') ('best', 'stuff') ('cheap', 'stuff') ('like', 'stuff') ('real', 'stuff') ('spicy